 ## THIRUMALA LAXMI TANUJA


In [1]:
# Download the data locally

import os
from urllib.request import urlretrieve
import gzip

# download the data if we don't have it locally
url = "http://faculty.cs.niu.edu/~dakoop/cs640-2023sp/a1/aam-salary-survey.csv.gz"
local_fname = "aam-salary-survey.csv.gz"
if not os.path.exists(local_fname):
    urlretrieve(url, local_fname)

# 1. Pandas

## a. Maximum Number of States Selected

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataframe = pd.read_csv("aam-salary-survey.csv.gz", compression = 'gzip')
df = dataframe.copy()
df['State_count'] = df['State'].apply(lambda x : len(x.split(',')) if isinstance(x, str) else np.nan)
max_states_index = df['State_count'].idxmax()
print("The job title and salary associated with the entry that has the most states selected are shown below")
print(f"Job Title : {df.loc[max_states_index]['JobTitle']}")
print(f"Salary    : {df.loc[max_states_index]['Salary']}")

The job title and salary associated with the entry that has the most states selected are shown below
Job Title : Project Controls Consultant
Salary    : 125,000


In [7]:
df.loc[['Timestamp', 'Age']]

KeyError: "None of [Index(['Timestamp', 'Age'], dtype='object')] are in the [index]"

## b. Number of Participants Working in Illinois 

In [49]:
df['State'] = df['State'].str.split(',')
df_exploded = df.explode('State')
participant_count = df_exploded[df_exploded['State'].str.strip() == 'Illinois'].shape[0]
print(f"Number of participants working in Illinois is {participant_count}")

Number of participants working in Illinois is 1216


## c. Highest US Salary 

In [50]:
df['Salary'] = df['Salary'].str.replace(',', "").astype(int)
df[df['Currency'] == "USD"]['Salary'].max()

102000000

## d. Latest Entry in 2021 

In [51]:
df = df.assign(TimeStampObject = pd.to_datetime(df["Timestamp"]))
index = df[df["TimeStampObject"].dt.year == 2021]["TimeStampObject"].idxmax()
df.loc[index]['Salary']

85000

## e. Top 10 Ways to Identify the U.S.

In [52]:
us_counts = df[df['State'].notnull()]['Country'].value_counts()
us_counts.head(10)

United States               8885
USA                         7851
US                          2585
United States                657
U.S.                         571
USA                          464
Usa                          443
United States of America     424
United states                205
usa                          182
Name: Country, dtype: int64

# 2. DuckDB

## a. Maximum Number of States Selected 

In [68]:
import duckdb
con = duckdb.connect(database=':memory:')
con.execute("CREATE TABLE salarysurvey AS SELECT * FROM read_csv_auto('aam-salary-survey.csv.gz', COMPRESSION ='gzip' )")
query = """ 

WITH temp_table AS (SELECT JobTitle, Salary, str_split(State, ',') as states_list FROM salarysurvey)
SELECT  arg_max([JobTitle,Salary], len(states_list)) FROM temp_table
"""
res = con.execute(query).fetchall()
res

[('[Project Controls Consultant, 125,000]',)]

## b. Number of Participants Working in Illinois

In [54]:
query = """SELECT COUNT(*) as IL_partcipants
FROM (
  SELECT UNNEST(str_split(State, ',')) AS state
  FROM salarysurvey
) 
WHERE state LIKE '%Illinois%';

"""
res = con.execute(query).fetch_df()
res

,IL_partcipants
0,1216


## c. Highest US Salary

In [55]:
query = """ SELECT MAX(Salary) AS highest_salary 
FROM (
SELECT TRY_CAST(replace(Salary, ',', '') AS INTEGER) AS Salary, Currency 
FROM salarysurvey
)
WHERE Currency = 'USD' """ 
res = con.execute(query).fetch_df()
res

,highest_salary
0,102000000


## d. Latest Entry in 2021

In [56]:
query = """
WITH temp_table AS 
    (
    SELECT Salary, strptime(Timestamp, '%m/%d/%Y %H:%M:%S') AS datatime 
    FROM salarysurvey
    )
SELECT Salary
FROM temp_table
WHERE datatime = (SELECT MAX(datatime)
FROM temp_table
WHERE EXTRACT(year FROM datatime) = 2021)
"""
res = con.execute(query).fetch_df()
res

,Salary
0,85000


## e. Top 10 Ways to Identify the U.S. 

In [57]:
query = """
SELECT Country, COUNT(Country) as Total_count
FROM salarysurvey
WHERE State != ''
GROUP BY Country
ORDER BY Total_count DESC
LIMIT 10
"""
res = con.execute(query).fetch_df()
res

,Country,Total_count
0,United States,8885
1,USA,7851
2,US,2585
3,United States,657
4,U.S.,571
5,USA,464
6,Usa,443
7,United States of America,424
8,United states,205
9,usa,182


## Ibis

## a. Maximum Number of States Selected

In [58]:
import duckdb
import ibis
t = ibis.read_csv("aam-salary-survey.csv.gz")
# t

In [59]:
ibis.options.interactive = True

In [60]:
# max_length = t.State.split(',').length()
# max_indices = max_length.argmax(key = max_length)
# max_indices = max_length.max()
# max_length
# res = t.filter(max_length == max_indices )
# res['JobTitle', 'Salary'][:1]

In [61]:
data = t.mutate(State = t.State.split(',').length())
print(data.JobTitle.argmax(data.State))
print(data.Salary.argmax(data.State))

'Project Controls Consultant'



'125,000'



## b. Number of Participants Working in Illinois

In [62]:
states = t.State.split(',').unnest().strip().name('states')
sample_table = states.value_counts()
sample_table.filter(sample_table.states == "Illinois")

┏━━━━━━━━━━┳━━━━━━━┓
┃ states   ┃ count ┃
┡━━━━━━━━━━╇━━━━━━━┩
│ string   │ int64 │
├──────────┼───────┤
│ Illinois │  1216 │
└──────────┴───────┘

## c. Highest US Salary 

In [63]:
t_usd = t.filter(t.Currency == 'USD')
t_usd.Salary.replace(",", "").cast('int64').max()

102000000

## d. Latest Entry in 2021

In [66]:
t = t.mutate(dateTimeObject = t.Timestamp.to_timestamp("%m/%d/%Y %H:%M:%S"))
t_2021 = t.filter(t["dateTimeObject"].year() == 2021)
# max_time_stamp = t_2021.dateTimeObject.argmax(t_2021.dateTimeObject)
# t_2021.filter(t_2021.dateTimeObject == max_time_stamp)["Salary"]
t_2021.Salary.argmax(t_2021.dateTimeObject)

'85000'

## e. Top 10 Ways to Identify the U.S.

In [65]:
us_idf_table = t.filter(t.State != "")
top_us_identifiers = us_idf_table.Country.value_counts()
top_us_identifiers = top_us_identifiers.order_by([ibis.desc('count'), (top_us_identifiers.Country, False)]).limit(10)
top_us_identifiers

┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Country                  ┃ count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ string                   │ int64 │
├──────────────────────────┼───────┤
│ United States            │  8885 │
│ USA                      │  7851 │
│ US                       │  2585 │
│ United States            │   657 │
│ U.S.                     │   571 │
│ USA                      │   464 │
│ Usa                      │   443 │
│ United States of America │   424 │
│ United states            │   205 │
│ usa                      │   182 │
└──────────────────────────┴───────┘